# Packages

In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import yfinance as yf
import yahoofinancials
from scipy.optimize import minimize
import seaborn as sn
from scipy.stats import anderson,kstest,shapiro, probplot

# Masalah.  
Misalkan portofolio saham dengan komposisi saham TLKM, BBNI, dan ASII. Akan dicari nilai return dengan variansi minimal dan juga dicari komposisi dari masing-masing saham.  

In [23]:
stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
        'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
#stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()

In [24]:
mu = df.mean().values
C = df.cov().values
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        {"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [25]:
np.sum(np.round(x_,5))

1.0

In [26]:
port_ = dict(zip(stock,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
port

{'BBNI.JK': 0.13289,
 'ANTM.JK': 0.00135,
 'ARTO.JK': 0.219,
 'AMRT.JK': 0.03105,
 'HRUM.JK': 0.14296,
 'INCO.JK': 0.07893,
 'ADRO.JK': 0.39382}

In [27]:
start = 2e7 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget_

{'BBNI.JK': 2657800.0,
 'ANTM.JK': 27000.0,
 'ARTO.JK': 4380000.0,
 'AMRT.JK': 621000.0,
 'HRUM.JK': 2859200.0,
 'INCO.JK': 1578600.0,
 'ADRO.JK': 7876400.0}

In [28]:
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
budget

,BBNI.JK,ANTM.JK,ARTO.JK,AMRT.JK,HRUM.JK,INCO.JK,ADRO.JK
Total Spread,2657800.0,27000.0,4380000.0,621000.0,2859200.0,1578600.0,7876400.0


In [29]:
expense = pd.DataFrame(columns=port.keys())
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
expense

,BBNI.JK,ANTM.JK,ARTO.JK,AMRT.JK,HRUM.JK,INCO.JK,ADRO.JK
Date,,,,,,,
Closing 2022-05-20,877500.0,249000.0,837500.0,177500.0,1105000.0,797500.0,331000.0


In [30]:
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,BBNI.JK,ANTM.JK,ARTO.JK,AMRT.JK,HRUM.JK,INCO.JK,ADRO.JK
Closing 2022-05-20,877500.0,249000.0,837500.0,177500.0,1105000.0,797500.0,331000.0
Total Spread,2657800.0,27000.0,4380000.0,621000.0,2859200.0,1578600.0,7876400.0
Total Lot,3.0,0.0,5.0,3.0,3.0,2.0,24.0
Total Spending,2632500.0,0.0,4187500.0,532500.0,3315000.0,1595000.0,7944000.0


In [31]:
finale.sum(axis = 1)

Closing 2022-05-20     4375000.0
Total Spread          20000000.0
Total Lot                   40.0
Total Spending        20206500.0
dtype: float64

In [32]:
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

return : 6.000000000614168 %
risk : 0.6088001251300724 %
Total Money : 20206500.0
Total Idle Money : -206500.0


In [33]:
finale.to_csv(str('IHSG '+finale.index[0].split(' ')[1]+'.csv'))

# Testing Profit

In [34]:
year = 2010
ds = str(year-1)+'-12-01'
de = str(year+2)+'-12-01'

stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
        'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
#stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()
df.dropna(axis = 1, inplace = True)

- ARTO.JK: Data doesn't exist for startDate = 1259600400, endDate = 1354294800


In [35]:
mu = df.mean().values
C = df.cov().values

def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [36]:
print(np.sum(np.round(x_,5)))
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

1.0
return : 0.39857247500472737 %
risk : 0.016070701505439555 %
Total Money : 20206500.0
Total Idle Money : -206500.0


In [37]:
port_ = dict(zip(df.columns.values,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
start = 2e7 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
expense = pd.DataFrame(columns=port.keys())

bs = str(year+2)+'-12-31'
be = str(year+3)+'-01-01'
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,TLKM.JK,BBNI.JK
Closing 2013-01-01,155220.898438,1.390727e+05
Total Spread,958800.000000,1.904120e+07
Total Lot,6.000000,1.370000e+02
Total Spending,931325.000000,1.905296e+07


In [38]:
buy = finale.columns.values
bs = str(year+2)+'-12-31'
be = str(year+4)+'-01-01'
profit_test = pd.DataFrame()
for item in buy:
    temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
    temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
    profit_test['buy_'+item] = temp['buy_'+item].dropna()
    profit_test['sell_'+item] = temp['sell_'+item].dropna()
    profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
profit_test.dropna(inplace = True)
profit_test


,buy_TLKM.JK,sell_TLKM.JK,return_TLKM.JK,buy_BBNI.JK,sell_BBNI.JK,return_BBNI.JK
Date,,,,,,
2013-02-01,9.265248e+05,1.032139e+06,105614.224655,1.905296e+07,1.905296e+07,0.000000e+00
2013-03-01,1.041740e+06,1.056142e+06,14401.938588,1.905296e+07,1.905296e+07,0.000000e+00
2013-04-01,1.056142e+06,1.123351e+06,67209.057617,1.905296e+07,5.501926e+07,3.596629e+07
2013-05-01,1.123351e+06,1.060943e+06,-62408.397001,5.621018e+07,5.074531e+07,-5.464879e+06
2013-06-01,1.071542e+06,1.086023e+06,14480.301758,5.100553e+07,4.475996e+07,-6.245575e+06
2013-07-01,1.061889e+06,1.148771e+06,86881.810498,4.502019e+07,4.449972e+07,-5.204646e+05
2013-08-01,1.153597e+06,1.061889e+06,-91708.579767,4.475995e+07,4.007577e+07,-4.684181e+06
2013-09-01,1.061889e+06,1.013621e+06,-48267.675781,3.955531e+07,4.241787e+07,2.862555e+06
2013-10-01,1.013621e+06,1.134290e+06,120669.192050,4.267810e+07,4.996460e+07,7.286505e+06


In [39]:
cols = profit_test.columns
finale_pt = pd.DataFrame()
tspend = 0
tprofit = 0 
for k in range (0,profit_test.shape[1], 3):
    tspend += profit_test[cols[k]]
    tprofit += profit_test[cols[k+2]]
finale_pt['Total Spending'] = tspend
finale_pt['Total Profit'] = tprofit
finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
finale_pt

,Total Spending,Total Profit,Total Return
Date,,,
2013-02-01,1.997949e+07,1.056142e+05,0.005286
2013-03-01,2.009470e+07,1.440194e+04,0.000717
2013-04-01,2.010911e+07,3.603350e+07,1.791900
2013-05-01,5.733354e+07,-5.527287e+06,-0.096406
2013-06-01,5.207707e+07,-6.231095e+06,-0.119651
2013-07-01,4.608208e+07,-4.335828e+05,-0.009409
2013-08-01,4.591355e+07,-4.775890e+06,-0.104019
2013-09-01,4.061720e+07,2.814288e+06,0.069288
2013-10-01,4.369172e+07,7.407174e+06,0.169533


In [40]:
print(f'Keuntungan dalam satu tahun pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

Keuntungan dalam satu tahun pada tahun 2013 sebesar: 13.99323662688504 %


# Loop

In [21]:
year = 2010
for n in range(8):
    year += 1
    ds = str(year-1)+'-12-01'
    de = str(year+2)+'-12-01'

    stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
            'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
    #stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
    df = pd.DataFrame(columns=stock)
    for i in stock :
        temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
        temp['log_return'].dropna(inplace = True)
        df[i] =temp['log_return'].dropna()
    df.dropna(axis = 1, inplace = True)

    mu = df.mean().values
    C = df.cov().values

    def objective(weights): 
        weights = np.array(weights)
        return weights.dot(C).dot(weights.T)
    # The constraints
    cons = (# The weights must sum up to one.
            {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
            # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
            #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

            )
    # Every stock can get any weight from 0 to 1
    bounds = tuple((0,1) for x in range(mu.shape[0]))
    # Initialize the weights with an even split
    # In out case each stock will have 10% at the beginning
    guess = [1./mu.shape[0] for x in range(mu.shape[0])]
    optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
    x_ = optimized_results.x
    print(np.sum(np.round(x_,5)))
    print(f'return :', x_.dot(mu)*100, '%')
    print(f'risk :', x_.dot(C).dot(x_)*100, '%')
    print(f'Total Money :',finale.sum(axis=1)[-1])
    print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])
    port_ = dict(zip(df.columns.values,np.round(x_,5)))
    port = dict()
    for (key,value) in port_.items():
        if value >0 : 
            port[key] = value
    start = 2e7 
    budget_ = dict()
    for (key,value) in port.items():
        if value >0 : 
            budget_[key] = value*start
    budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
    budget.rename(index={0: 'Total Spread'}, inplace= True)
    expense = pd.DataFrame(columns=port.keys())

    bs = str(year+2)+'-12-31'
    be = str(year+3)+'-01-01'

    for item in port.keys(): 
        temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
        expense[item] = temp['Close']*100 #1 lot
    expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
    qty = pd.concat([expense,budget])
    temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
    temp_.rename(index={0: 'Total Lot'}, inplace= True)
    qty = pd.concat([qty, temp_])
    spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
    finale = pd.concat([qty, spend])
    finale.rename(index={0: 'Total Spending'}, inplace= True)
    finale
    buy = finale.columns.values
    bs = str(year+2)+'-12-31'
    be = str(year+4)+'-01-01'
    profit_test = pd.DataFrame()
    for item in buy:
        temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
        temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
        profit_test['buy_'+item] = temp['buy_'+item].dropna()
        profit_test['sell_'+item] = temp['sell_'+item].dropna()
        profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
    profit_test.dropna(inplace = True)
    cols = profit_test.columns
    finale_pt = pd.DataFrame()
    tspend = 0
    tprofit = 0 
    for k in range (0,profit_test.shape[1], 3):
        tspend += profit_test[cols[k]]
        tprofit += profit_test[cols[k+2]]
    finale_pt['Total Spending'] = tspend
    finale_pt['Total Profit'] = tprofit
    finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
    finale_pt.to_csv('Profit Test' +str(year)+'.csv')
    print(f'Rata-rata satu tahun keuntungan perbulan pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

- ARTO.JK: Data doesn't exist for startDate = 1291136400, endDate = 1385830800
1.0
return : 0.5453088014986209 %
risk : 0.13392051731923538 %
Total Money : 19984291.0
Total Idle Money : 15709.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2014 sebesar: 1.210178669153215 %
- ARTO.JK: Data doesn't exist for startDate = 1322672400, endDate = 1417366800
1.0
return : 0.8671123623191029 %
risk : 0.08397377566071088 %
Total Money : 19474932.0
Total Idle Money : 525068.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2015 sebesar: -1.4894874368727429 %
- ARTO.JK: Data doesn't exist for startDate = 1354294800, endDate = 1448902800
1.0
return : 0.42655365817823804 %
risk : 0.1098631254359246 %
Total Money : 19815569.0
Total Idle Money : 184431.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2016 sebesar: 3.589437640708633 %
1.0000000000000002
return : 1.2310250423272722 %
risk : 0.08191485049900334 %
Total Money : 20149815.0
Total Idle Money : -149815.0
Rata-rata satu tahun keu